# ChromaDB 생성

In [ ]:
import os
import glob
import re
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# PDF 및 TXT 폴더 경로
pdf_directory = "./pdf"
txt_directory = "./txt"

documents = []

# PDF 파일 로드
pdf_files = glob.glob(os.path.join(pdf_directory, "*.pdf"))
print(f"{len(pdf_files)}개의 PDF 파일을 발견했습니다.")
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())

# TXT 파일 로드
txt_files = glob.glob(os.path.join(txt_directory, "*.txt"))
print(f"{len(txt_files)}개의 TXT 파일을 발견했습니다.")
for txt_file in txt_files:
    loader = TextLoader(txt_file, encoding='utf-8')
    documents.extend(loader.load())

print(f"총 {len(documents)}개의 문서 로드 완료")

# 텍스트 전처리 함수
def clean_text(text: str) -> str:
    text = text.replace('\xa0', ' ')
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,!?()\[\]{}가-힣A-Za-z0-9]', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text) # 중복 문자 정규화
    text = re.sub(r'\([^)]*\)', '', text) # 괄호 및 내용 제거
    return text.strip()

# 문서 전처리 적용
for i, doc in enumerate(documents):
    documents[i].page_content = clean_text(doc.page_content)

# 텍스트 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
)

chunks = text_splitter.split_documents(documents)
chunks = [chunk for chunk in chunks if len(chunk.page_content) > 10]  # 빈 청크 제거
print(f"총 {len(chunks)}개의 청크로 분할 완료")

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# ✅ Chroma 벡터 저장소 생성 및 저장
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_index_400_100"
)

23개의 PDF 파일을 발견했습니다.


KeyboardInterrupt: 

# 크로마 적용

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# 임베딩 및 벡터스토어 로드
embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
vector_store = Chroma(
    persist_directory="chroma_index_300_100",
    embedding_function=embedding
)
retriever = vector_store.as_retriever()

/var/folders/qv/kf5vm6ln5k113k6v46_6ynn00000gn/T/ipykernel_63355/4268528185.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
/opt/anaconda3/envs/vectordb_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/qv/kf5vm6ln5k113k6v46_6ynn00000gn/T/ipykernel_63355/4268528185.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0.